# Analítica y Ciencia de Datos

## CIDE - Otoño 2015

### SOLUCION: Tarea 4: Introducción al Aprendizaje Estadístico

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Necesitamos nuestra función de OLS.
# vamos a crear una función OLS para utilizar varias veces
def olsfun(y,X):
    '''
    ------------------
    Objetivo: estimar los coeficientes de un modelo de regresión lineal por medio de mínimos cuadrados
    ordinarios (OLS).
    ------------------
    Argumentos: 
    y: Variable dependiente: de tamaño Nx1
    X: matriz de variables independientes: de tamaño NxK
    ------------------
    Returns:
    betahat: vector de coeficientes estimados
    mse    : error cuadrático medio muestral
    vcv    : matriz de varianzas y covarianzas de los coeficientes
    yhat   : y estimado
    ------------------
    '''
    N,K = X.shape
    # necesitamos la matriz inversa
    xtx = np.dot(X.T, X)
    xtxinv = np.linalg.inv(xtx)
    xty = np.dot(X.T,y)
    betahat = np.dot(xtxinv,xty)
    # obtengamos también el MSE
    yhat = np.dot(X,betahat)
    resids = y-yhat
    ssr  = np.dot(resids.T,resids)
    s2   = ssr/(N-K)
    mse  = ssr/N
    # la matriz de VCV
    vcv  = s2*xtxinv 
    return betahat, mse[0,0], vcv, yhat

## Problema 1: Método Delta

**Parte 1. Loglinealícelo**:

Lo hicimos en las notas:

$$
\ln(y) = \beta_0 + \beta_1 \ln(x) + \epsilon
$$

**Parte 2. Qué parámetros podemos estimar:**

El modelo es lineal en $\beta_0$ y $\beta_1$, así que los podemos estimar con OLS.  Los parámetros profundos $\alpha_0$ y $\alpha_1$ no se pueden estimar directamente.

Una vez tengamos $\hat{\beta_i}$, podemos obtener $\hat{\alpha_i}$ mediante las transformacinoes correspondientes:

$$
\beta_0 = \ln(\alpha_0)  \Longrightarrow \alpha_0 = e^{\beta_0}\\
\beta_1 = 1/\alpha_1     \Longrightarrow \alpha_1 = 1/\beta_1
$$

**Parte 3: Método Delta**

Como vimos en las Notas:

Si $\alpha = f(\beta)$, y $\beta \sim N(\mu_b,\sigma_b^2)$

$$
f(\beta) \approx f(\beta_0) + f'(\beta_0)(\beta - \beta_0)
$$
así que, si denotamos a $\alpha_0 := f(\beta_0)$
$$
\alpha - \alpha_0 \sim^{d} N(0, f'(\beta_0)^2 \sigma_b^2)
$$

(para los interesados $\sim^d$ significa convergencia en distribución, aunque una explicación de esto nos lleva por fuera del tema del curso)

* En nuestro caso: $\alpha_1 = f(\beta_1) = 1/\beta_1$, así que

$$
\alpha_1 - \alpha^*_{1} \sim^d N(0, (1/\beta_1)^2 \sigma_{b1}^2)
$$

donde las $*$ denotan al parámetro verdadero.

* Y para la constante: $\alpha_0 = e^{\beta_0}$, así que

$$
\alpha_0 - \alpha^*_{0} \sim^d N(0,  e^{2\beta_0} \sigma_{b0}^2)
$$

Hasta aquí la teoría.  En la práctica el procedimiento es trivial:

1. Estimar el modelo loglineal, y obtener las varianzas de los parámetros $\hat{\beta}_0, \hat{\beta}_1$.
2. Por lo anterior sabemos que:

$$
\begin{eqnarray*}
Var(\hat{\alpha}_0) &=& e^{2\hat{\beta}_0} \hat{\sigma}_{b0}^2\\
Var(\hat{\alpha}_1) &=& (1/\hat{\beta}_1)^2 \hat{\sigma}_{b1}^2
\end{eqnarray*}
$$


**Parte 4: Simule el modelo y estime las varianzas del punto anterior**

In [23]:
np.random.seed(19790625)
N = 300
alpha0 = 10
alpha1 = 2.0
# para que sea positivo, desplacémoslo
x      = 0.1*np.random.randn(N,1)
epsilon = np.sqrt(0.5)*np.random.randn(N,1)
# listos para estimar el modelo:
beta1  = 1/alpha1
y = alpha0*np.power(x,beta1)*np.exp(epsilon)
# Eliminemos los NaNs que se obtendrán cuando hagamos ln(x)
data = pd.DataFrame(np.concatenate((y,np.log(x)),axis=1), columns=['y','x']).dropna(subset= ['x'])
N = data.shape[0]
y = np.asarray(data.y).reshape((N,1))
x = np.asarray(np.exp(data.x.values)).reshape((N,1))
# El modelo a estimar loglineal:
lny = np.log(y)
lnx = np.log(x)
xmat = np.concatenate((np.ones((N,1)),lnx),axis=1)
# estimemos
resols = olsfun(lny,xmat)
# Nos interesan las varianzas
betahat = resols[0]
vcv     = resols[2]
print betahat
print vcv
# Obtengamos los parámetros estructurales:
alpha0hat = np.exp(betahat[0])
alpha1hat = 1/betahat[1]
print "----------------"
print "alpha0 verdadero y el estimado son: ", (alpha0,alpha0hat[0])
print "alpha1 verdadero y el estimado son: ", (alpha1,alpha1hat[0])
print "----------------"
# finalmente, las varianzas:
var_a0 = np.exp(2*betahat[0][0])*vcv[0,0]
var_a1 = ((1/betahat[1][0])**2)*vcv[1,1]
print "la varianza de alpha0 = ", var_a0
print "la varianza de alpha1 = ", var_a1

[[ 2.21867885]
 [ 0.46153297]]
[[ 0.03523143  0.01086236]
 [ 0.01086236  0.00376306]]
----------------
alpha0 verdadero y el estimado son:  (10, 9.1951746326739574)
alpha1 verdadero y el estimado son:  (2.0, 2.1666924372982828)
----------------
la varianza de alpha0 =  2.97886075138
la varianza de alpha1 =  0.0176659180991


**Parte 5: comparación con Bootstrapping.**

En las notas de clase vimos que 
$$
\begin{eqnarray*}
Var(\hat{\alpha}_0^2) &=& 0.00389043\\
Var(\hat{\alpha}_1^2) &=& 0.09927349\\
\end{eqnarray*}
$$

La varianza de $\alpha_1$ son comparables en términos de órdenes de magnitud, pero no es así en el caso de $\alpha_0$, donde bootstrapping produce una varianza considerablemente más baja.

Lo importante no es la varianza, per sé, sino los efectos que esto tiene sobre los resultados de las pruebas de hipótesis.

En particular, queremos que, en promedio, los dos métodos permitan rechazar la hipótesis nula de ausencia de los efectos con la misma probabilidad, pero esto sólo podríamos saberlo con un experimento de Monte Carlo, en donde simulemos el modelo $K$ veces, y obtengamos la fracción de veces en que los correspondientes p-values permiten rechazar $H_0$.  Sólo así podríamos saber si los métodos producen resultados comparables y cuál es mejor o peor.

No obstante, lo que sí podemos decir ya es lo siguiente:

> El Método Delta hace una aproximación de Taylor de primer orden, o lineal, así que la calidad de la misma depende de qué tan no lineal es el problema.  Este no es el caso con Bootstrapping, así que en general, esta técnica es preferible.



# Problema 2: Jackknife

Como vimos en las notas de clase, tenemos que hacer N estimaciones, excluyendo cada observación

$$
\begin{array}{l|l}
iter & muestra \\
\hline
0    & 1,2,\cdots,N-1 \\
1    & 0,2,\cdots,N-1 \\
2    & 0,1,3,\cdots,N-1 \\
\vdots    & \vdots \\
N-1  & 0,1,2,\cdots, N-2\\
\hline
\end{array}
$$


In [30]:
# Inicialicemos donde vamos a guardar los resultados
JKEstimator = np.zeros((N,2))
# podemos empezar la iteración
for i in range(N):
    # para excluir la i-ésima observación podemos utilizar la función setdiff1d, por ejemplo
    ind_inc = np.setdiff1d(np.arange(N), np.array([i]))
    # listo para seleccionar los datos:
    y_i = np.log(y[ind_inc]).reshape((N-1,1))
    x_i = np.log(x[ind_inc]).reshape((N-1,1))
    # Estimemos el modelo
    olsi = olsfun(y_i, np.concatenate((np.ones((N-1,1)),x_i),axis=1))
    betai = olsi[0].flatten()
    # Guardemos los resultados:
    JKEstimator[i,:] = betai
# Listos, obtengamos la varianza:
JKVar = np.var(JKEstimator,axis=0)
print JKVar

[  2.72414413e-04   3.05673761e-05]


La varianza es aún menor que la que estimamos con Bootstrapping: [ 0.00389043  0.09927349]

# Problema 3: Validación Cruzada

In [46]:
# Es muy simple:
N = 100
grp_ind = np.dot(np.random.multinomial(1,[1.0/5]*5,N),np.arange(1,6).reshape(5,1)).flatten()
# Revisemos que quedó bien hecho con un groupby
pd.Series(grp_ind).groupby(pd.Series(grp_ind)).count()

1    24
2    20
3    18
4    25
5    13
dtype: int64